In [5]:
# Import required Packages

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup
import pandas as pd

In [14]:
# Declare constants
training_source = 'ACloudGuru'
no_of_courses = 3

# course id
aws_developer_associate = 0
aws_soultions_architect_associate = 1
aws_certified_cloud_practitioner = 2

# Course names
exam_names = ['']*no_of_courses
exam_names[aws_developer_associate] ='aws-developer-associate'
exam_names[aws_soultions_architect_associate] ='aws-soultions-architect-associate' 
exam_names[aws_certified_cloud_practitioner] ='aws-certified-cloud-practitioner' 

# Exam Codes
exam_codes = ['']*no_of_courses
exam_codes[aws_developer_associate] ='dva-c01'
exam_codes[aws_soultions_architect_associate] ='saa-c02' 
exam_codes[aws_certified_cloud_practitioner] ='clf-c01' 

# Course urls
urls =['']*no_of_courses
urls[aws_developer_associate] = 'https://acloud.guru/learn/aws-certified-developer-associate'
urls[aws_soultions_architect_associate] ='https://acloud.guru/learn/aws-certified-solutions-architect-associate'
urls[aws_certified_cloud_practitioner]='https://acloud.guru/learn/aws-certified-cloud-practitioner'

In [15]:
# Set Current Course & URL 
current_exam_scraped = aws_soultions_architect_associate
exam_code = exam_codes[current_exam_scraped]
exam_name = exam_names[current_exam_scraped]
start_url = urls[current_exam_scraped]
tech_name = 'aws'

file_name = training_source+'-'+exam_name

In [16]:
# Navigate to page with course contents and grab the page
with webdriver.Firefox() as driver:
    wait = WebDriverWait(driver, 10)
    driver.get(start_url)

    time.sleep(25)
 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(3)


    # And grab the page HTML source
    html_page = driver.page_source
    
    # Save a local Copy
    with open(file_name+'.html','w') as file:
        file.write(html_page)
          

In [17]:
# Read the Saved HTML File 
source_page =''   
# Read the local saved copy
with open(file_name+'.html','r') as file:
    source_page = file.read()  

In [18]:
# Start Processing the file
# Parse the HTML with BeautifulSoup and create a soup object
soup = BeautifulSoup(source_page)

table_columns = []
table_columns.append('exam code')
table_columns.append('tech')
table_columns.append('training_source')
table_columns.append('exam_name')
table_columns.append('course_name')
table_columns.append('title_order')
table_columns.append('title')
table_columns.append('lectures')
table_columns.append('sub_section')
table_columns.append('duration')

table_data = []
title_order = 0

# Grap the course name 
course_name = soup.select('div[class="col-md-6 course-preview--jumbotron-info"]')[0].select('h1')[0].text


# Select course list
course_content = soup.select('div[class="curriculum-section"]') 
for ele in course_content:
    row_data = []
    title_order = title_order + 1
    no_of_lectures= ''
    
    # Extract the data
    heading = ele.select('h2[class="curriculum--heading"]')
    duration = heading[0].select('span')[0].text
    title = heading[0].text.replace(duration,'')
    
    # Set the row data
    row_data.append(exam_code)
    row_data.append(tech_name)
    row_data.append(training_source)
    row_data.append(exam_name)
    row_data.append(course_name)
    row_data.append(title_order)
    row_data.append(title)
    row_data.append(no_of_lectures)
    row_data.append('')
    row_data.append(duration)
    table_data.append(row_data)
   

    sub_sections = ele.select('span[class="curriculum--metadata-title"]')
    sub_sections_count = 0
    for index,sub_title in enumerate(sub_sections):
        sub_sections_count +=1
        sub_row = row_data.copy()
        
        sub_duration = ele.select('span[class="duration"]')[index]
        sub_row[8] = sub_title.text
        sub_row[9] = sub_duration.text
        table_data.append(sub_row)
    row_data[7] =sub_sections_count
 

In [19]:
# Create a Pandas DataFrame
df = pd.DataFrame(table_data, columns=table_columns)
df.to_excel(file_name+".xlsx") 
df

,exam code,tech,training_source,exam_name,course_name,title_order,title,lectures,sub_section,duration
0,saa-c02,aws,ACloudGuru,aws-soultions-architect-associate,AWS Certified Solutions Architect Associate SA...,1,Chapter 1 Introduction,3,,19:49
1,saa-c02,aws,ACloudGuru,aws-soultions-architect-associate,AWS Certified Solutions Architect Associate SA...,1,Chapter 1 Introduction,,Introduction,01:19
2,saa-c02,aws,ACloudGuru,aws-soultions-architect-associate,AWS Certified Solutions Architect Associate SA...,1,Chapter 1 Introduction,,The Exam Blueprint,05:09
3,saa-c02,aws,ACloudGuru,aws-soultions-architect-associate,AWS Certified Solutions Architect Associate SA...,1,Chapter 1 Introduction,,Why Should I Learn and Get Certified with AWS?,13:21
4,saa-c02,aws,ACloudGuru,aws-soultions-architect-associate,AWS Certified Solutions Architect Associate SA...,2,"Chapter 2 AWS - 10,000-Foot Overview",4,,40:17
...,...,...,...,...,...,...,...,...,...,...
160,saa-c02,aws,ACloudGuru,aws-soultions-architect-associate,AWS Certified Solutions Architect Associate SA...,13,Chapter 13 Good Luck!,,Good Luck and How to Book Your Exam,03:29
161,saa-c02,aws,ACloudGuru,aws-soultions-architect-associate,AWS Certified Solutions Architect Associate SA...,13,Chapter 13 Good Luck!,,Thank You and Next Steps,01:26
162,saa-c02,aws,ACloudGuru,aws-soultions-architect-associate,AWS Certified Solutions Architect Associate SA...,13,Chapter 13 Good Luck!,,AWS Certified Solutions Architect Associate 20...,15:00
163,saa-c02,aws,ACloudGuru,aws-soultions-architect-associate,AWS Certified Solutions Architect Associate SA...,13,Chapter 13 Good Luck!,,AWS Certified Solutions Architect Associate 20...,15:00
